# 🌐 MNIST Web App for Kaggle

This notebook creates a simple web interface using Gradio to test your MNIST model on Kaggle.

**Note:** This works great on Kaggle and doesn't require GPU for inference!


In [ ]:
# Install Gradio (if not already installed)
%pip install gradio -q


In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import numpy as np
import gradio as gr

# Import your model
import sys
sys.path.append('/kaggle/working')
from model import MNISTCNN


In [ ]:
# Load your trained model
model_path = '/kaggle/working/models/mnist_cnn_model.pth'  # Adjust path as needed

model = MNISTCNN()
model.load_state_dict(torch.load(model_path, map_location='cpu'))
model.eval()

print("✅ Model loaded successfully!")


In [ ]:
def preprocess_image(image):
    """Preprocess image for MNIST model."""
    # Convert to grayscale if needed
    if isinstance(image, np.ndarray):
        if len(image.shape) == 3:
            # Convert RGB to grayscale
            image = Image.fromarray(image).convert('L')
        else:
            image = Image.fromarray(image)
    
    # Ensure it's a PIL Image
    if not isinstance(image, Image.Image):
        image = Image.fromarray(image)
    
    # Convert to grayscale
    image = image.convert('L')
    
    # Resize to 28x28
    image = image.resize((28, 28), Image.LANCZOS)
    
    # Convert to numpy array
    img_array = np.array(image)
    
    # Invert if needed (MNIST has white digits on black background)
    if img_array.mean() < 128:
        img_array = 255 - img_array
    
    # Normalize
    img_array = img_array.astype(np.float32) / 255.0
    img_array = (img_array - 0.1307) / 0.3081
    
    # Convert to tensor
    img_tensor = torch.from_numpy(img_array).unsqueeze(0)
    
    return img_tensor

def predict_digit(image):
    """Predict digit from uploaded image."""
    try:
        # Preprocess image
        img_tensor = preprocess_image(image)
        
        # Make prediction
        with torch.no_grad():
            output = model(img_tensor.unsqueeze(0))
            probabilities = F.softmax(output, dim=1)
            predicted = torch.argmax(output, dim=1).item()
            confidence = probabilities[0][predicted].item() * 100
        
        # Get all probabilities
        all_probs = probabilities[0].cpu().numpy() * 100
        
        # Create result string
        result = f"🎯 Predicted Digit: **{predicted}**\n\n"
        result += f"📊 Confidence: **{confidence:.2f}%**\n\n"
        result += "📈 Probability Distribution:\n"
        result += "\n".join([f"  {i}: {all_probs[i]:.2f}%" for i in range(10)])
        
        return result
    except Exception as e:
        return f"❌ Error: {str(e)}"

print("✅ Prediction function ready!")


In [ ]:
# Create Gradio interface
iface = gr.Interface(
    fn=predict_digit,
    inputs=gr.Image(type="pil", label="Upload Handwritten Digit"),
    outputs=gr.Textbox(label="Prediction Result"),
    title="🔢 MNIST Digit Classifier",
    description="Upload an image of a handwritten digit (0-9) and get instant predictions!",
    examples=None,
    theme=gr.themes.Soft()
)

# Launch the interface
# Use share=True to create a public link
# Use share=False for local access only
iface.launch(share=True, server_name="0.0.0.0", server_port=7860)


## 🎉 Your Web App is Live!

After running the last cell, you'll get:
- A public URL (if share=True)
- A local URL for testing

**Tips:**
- Upload clear images of handwritten digits
- Works best with white/light backgrounds
- The model automatically preprocesses your image

**Note:** On Kaggle, the public link will work as long as the notebook is running. For permanent hosting, consider deploying the Flask app to a cloud service.
